In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
import matplotlib.pyplot as plt

In [4]:
dfcases = pd.read_csv("https://api.vitaldb.net/cases")  # clinical information

In [5]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Initialize weights
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)

        # Initialize the biases
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.bias_output = np.zeros((1, self.output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def feedforward(self, X):
        # Input to hidden
        self.hidden_activation = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_activation)

        # Hidden to output
        self.output_activation = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = self.sigmoid(self.output_activation)

        return self.predicted_output

    def backward(self, X, y, learning_rate):
        # Compute the output layer error
        output_error = y.reshape(-1, 1) - self.predicted_output  # Ensure y matches the output dimensions
        output_delta = output_error * self.sigmoid_derivative(self.predicted_output)

        # Compute hidden layer error
        hidden_error = np.dot(output_delta, self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)

        # Update the weights and biases
        self.weights_hidden_output += np.dot(self.hidden_output.T, output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        self.weights_input_hidden += np.dot(X.T, hidden_delta) * learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate


    def train(self, X, y, epochs, learning_rate,conv=0.2):
        for epoch in range(epochs):
            output = self.feedforward(X)
            self.backward(X, y, learning_rate)
            loss = np.mean(np.square(y - output))
            if conv>loss:
                break




height = dfcases["height"].values
weight = dfcases["weight"].values
bmi = dfcases["bmi"].values
X = np.array([[y,x] for x,y in zip(height,weight)])
y = np.array([i for i in bmi])


nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)
nn.train(X, y, epochs=1000, learning_rate=0.1,conv=0.002)

output  = nn.feedforward(X)
print(output)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [9]:


param_distributions_mlp = {
    'hidden_layer_sizes': [(50,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.002, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [100, 500, 1000],
    'tol': [1e-3, 1e-4, 1e-5]
}


mlp = MLPClassifier(hidden_layer_sizes=(6,), activation='logistic', solver='lbfgs', max_iter=1000)



random_search_mlp = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_distributions_mlp,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    verbose=2,
    random_state=0
)

# Prepare your data - Ensure y is classification-friendly (binary/multi-class labels)
height = dfcases["height"].values
weight = dfcases["weight"].values
bmi = dfcases["bmi"].values
X = np.array([[y,x] for x,y in zip(height, weight)])
y = np.digitize(bmi, bins=[18.5, 24.9])  # Example: Converting BMI to categorical (binary)

random_search_mlp.fit(X, y)

# Print best hyperparameters and scores
print("Best Perceptron Hyperparameters:", random_search_perceptron.best_params_)
print("Best Perceptron Accuracy:", random_search_perceptron.best_score_)
print("\n")
print("Best MLP Hyperparameters:", random_search_mlp.best_params_)
print("Best MLP Accuracy:", random_search_mlp.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.0s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.0s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.0s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.0s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.0s
[CV] END ....alpha=0.01, max_iter=500, penalty=l2, tol=1e-05; total time=   0.0s
[CV] END ....alpha=0.01, max_iter=500, penalty=l

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=500, solver=sgd, tol=1e-05; total time=   7.5s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=500, solver=sgd, tol=1e-05; total time=   5.7s


...

[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=100, solver=sgd, tol=0.0001; total time=   1.2s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=100, solver=sgd, tol=0.0001; total time=   1.2s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=100, solver=sgd, tol=0.0001; total time=   1.7s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   1.9s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.9s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.4s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.3s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.8s
Best Perceptron Hyperparameters: {'tol': 1e-05, 'penalty': 'l2', 'max_iter': 100, 'alpha': 0.001}
Best Perceptron Accuracy: 0.7

In [5]:

height = dfcases["height"].values
weight = dfcases["weight"].values
bmi = dfcases["bmi"].values
X = np.array([[y,x] for x,y in zip(height,weight)])
y = np.array([i for i in bmi])
mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', max_iter=10000000000000000)
y = y.astype('int')
mlp.fit(X, y)
for i in range(len(X)//100):
    prediction = mlp.predict([X[i]])[0]
    print(f"Input: {X[i]}, Target: {y[i]}, Prediction: {prediction}")

Input: [ 67.5 160.2], Target: 26, Prediction: 25
Input: [ 54.8 167.3], Target: 19, Prediction: 19
Input: [ 69.7 169.1], Target: 24, Prediction: 24
Input: [ 53.  160.6], Target: 20, Prediction: 20
Input: [ 59.7 171. ], Target: 20, Prediction: 20
Input: [ 54.6 150. ], Target: 24, Prediction: 24
Input: [ 62.3 167.7], Target: 22, Prediction: 21
Input: [ 67.25 156.7 ], Target: 27, Prediction: 25
Input: [ 50.9 157.9], Target: 20, Prediction: 20
Input: [ 62.75 162.5 ], Target: 23, Prediction: 23
Input: [ 81.45 175.4 ], Target: 26, Prediction: 25
Input: [ 81.4 169.2], Target: 28, Prediction: 26
Input: [ 64.9 153. ], Target: 27, Prediction: 25
Input: [ 80.  177.9], Target: 25, Prediction: 25
Input: [ 48.3 158. ], Target: 19, Prediction: 19
Input: [ 68.9 162.3], Target: 26, Prediction: 25
Input: [ 53.  164.2], Target: 19, Prediction: 19
Input: [ 56.9 155. ], Target: 23, Prediction: 23
Input: [ 66.2 171.3], Target: 22, Prediction: 22
Input: [ 61.3 173.6], Target: 20, Prediction: 20
Input: [ 61.6 

# # A2. Use cross-validation techniques (RandomizedSearchCV()) technique to tune the hyperparameters for your perceptron and MLP networks.

In [14]:
def perceptron_tuning():
    param_distributions_perceptron = {
    'penalty': ['l2'],  # Perceptron only supports 'l2'
    'alpha': [0.001, 0.01, 0.1],  # Keep a reasonable range
    'max_iter': [100, 500, 1000],
    'tol': [1e-3, 1e-4, 1e-5]
    }
    perceptron = Perceptron(random_state=0)
    random_search_perceptron = RandomizedSearchCV(
    estimator=perceptron,
    param_distributions=param_distributions_perceptron,
    n_iter=10,  # Number of parameter settings to sample
    cv=5,  # Cross-validation folds
    scoring='accuracy',  # Evaluation metric
    verbose=2,  # Output verbosity level
    random_state=0
    )
    # Prepare your data - Ensure y is classification-friendly (binary/multi-class labels)
    height = dfcases["height"].values
    weight = dfcases["weight"].values
    bmi = dfcases["bmi"].values
    X = np.array([[y,x] for x,y in zip(height, weight)])
    y = np.digitize(bmi, bins=[18.5, 24.9])  # Example: Converting BMI to categorical (binary)

    # Fit the models with RandomizedSearchCV
    random_search_perceptron.fit(X, y)
    return random_search_perceptron.best_params_,random_search_perceptron.best_score_





In [17]:
def mlp_tuning():


    param_distributions_for_mlp = {
        'hidden_layer_sizes': [(50,)],
        'activation': ['relu', 'tanh', 'logistic'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'alpha': [0.002, 0.001, 0.01],
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
        'max_iter': [100, 500, 1000],
        'tol': [1e-3, 1e-4, 1e-5]
    }


    mlp = MLPClassifier(hidden_layer_sizes=(6,), activation='logistic', solver='lbfgs', max_iter=1000)



    random_search_mlp = RandomizedSearchCV(
        estimator=mlp,
        param_distributions=param_distributions_mlp,
        n_iter=10,
        cv=5,
        scoring='accuracy',
        verbose=2,
        random_state=0
    )

    # Prepare your data - Ensure y is classification-friendly (binary/multi-class labels)
    height = dfcases["height"].values
    weight = dfcases["weight"].values
    bmi = dfcases["bmi"].values
    X = np.array([[y,x] for x,y in zip(height, weight)])
    y = np.digitize(bmi, bins=[18.5, 24.9])  # Example: Converting BMI to categorical (binary)

    random_search_mlp.fit(X, y)
    return random_search_mlp.best_params_,random_search_mlp.best_score_



# Tabulate your results with various other classifiers such as Support Vector Machines, Decision Tree, RandomForest, CatBoost, AdaBoost, XGBoost, Naïve-Bayes. Tabulate your results for your problem using different performance metrics

In [23]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
def A3():
    height = dfcases["height"].values
    weight = dfcases["weight"].values
    bmi = dfcases["bmi"].values
    X = np.array([[y,x] for x,y in zip(height, weight)])
    y = np.digitize(bmi, bins=[18.5, 24.9])

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set up classifiers
    model_collection = {
        'Perceptron': Perceptron(**random_search_perceptron.best_params_, random_state=42),
        'MLP': MLPClassifier(**random_search_mlp.best_params_, random_state=42),
        'SVM': SVC(random_state=42),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'CatBoost': CatBoostClassifier(random_state=42, verbose=False),
        'AdaBoost': AdaBoostClassifier(random_state=42),
        'XGBoost': XGBClassifier(random_state=42, objective='multi:softprob'),
        'Naive Bayes': GaussianNB()
    }

    # Define etrics
    eval_metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']

    # Get results matrix
    results_df = pd.DataFrame(columns=['Model'] + eval_metrics)

    # Training every model
    for model_name, model in model_collection.items():

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        # Calculate performance stats
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='macro')
        rec = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')


        fresh_results = pd.DataFrame({'Model': [model_name],
                                      'Accuracy': [acc],
                                      'Precision': [prec],
                                      'Recall': [rec],
                                      'F1-score': [f1]})
        results_df = pd.concat([results_df, fresh_results], ignore_index=True)


    return results_df


In [24]:
def main():
    best_params, score = perceptron_tuning()
    print("Best Perceptron Hyperparameters:", best_params)
    print("Best Perceptron Accuracy:", score)

    best_params, score = mlp_tuning()
    print("Best MLP Hyperparameters:", best_params)
    print("Best MLP Accuracy:", score)
    results = A3()
    print(results)

In [25]:
main()

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.1s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.1s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.1s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.1s
[CV] END ...alpha=0.001, max_iter=100, penalty=l2, tol=1e-05; total time=   0.1s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.1s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.1s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.1s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.1s
[CV] END ....alpha=0.1, max_iter=1000, penalty=l2, tol=0.001; total time=   0.1s
[CV] END ....alpha=0.01, max_iter=500, penalty=l2, tol=1e-05; total time=   0.1s
[CV] END ....alpha=0.01, max_iter=500, penalty=l

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=500, solver=sgd, tol=1e-05; total time=  14.6s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=500, solver=sgd, tol=1e-05; total time=   5.8s


...

[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=100, solver=sgd, tol=0.0001; total time=   2.8s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=invscaling, max_iter=100, solver=sgd, tol=0.0001; total time=   1.2s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.4s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.5s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.7s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.4s
[CV] END activation=relu, alpha=0.002, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=500, solver=sgd, tol=1e-05; total time=   0.9s
Best MLP Hyperparameters: {'tol': 0.001, 'solver': 'lbfgs', 'max_iter': 500, 'learning_rate': 'invscaling', 'hidden_layer_sizes

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-23-abadb460d6b6>:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, fresh_results], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-l

           Model  Accuracy  Precision    Recall  F1-score
0     Perceptron  0.662754   0.548325  0.387205  0.354146
1            MLP  0.935837   0.914702  0.850884  0.877491
2            SVM  0.970266   0.972376  0.916647  0.941421
3  Decision Tree  0.987480   0.984362  0.982701  0.983518
4  Random Forest  0.992175   0.994932  0.979788  0.987137
5       CatBoost  0.989045   0.980866  0.986077  0.983438
6       AdaBoost  0.718310   0.862928  0.542068  0.583505
7        XGBoost  0.982786   0.972937  0.965440  0.969112
8    Naive Bayes  0.748044   0.720717  0.590909  0.628476


In [22]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00
